In [120]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression


In [121]:
#Cargar archivo csv
DF= pd.read_csv("/content/Mexico_City.csv", encoding= 'latin')
df = DF.copy()
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,35797.0,https://www.airbnb.com/rooms/35797,2.025060e+13,26/06/2025,city scrape,Villa Dante,"Dentro de Villa un estudio de arte con futon, ...","Santa Fe Shopping Mall, Interlomas Park and th...",https://a0.muscache.com/pictures/f395ab78-1185...,153786,...,4.84,4.82,4.84,4.71,f,1,1,0,0,1.81
1,44616.0,https://www.airbnb.com/rooms/44616,2.025060e+13,01/07/2025,city scrape,Condesa Haus,A new concept of hosting in mexico through a b...,No neighborhood overview provided,https://a0.muscache.com/pictures/251410/ec75fe...,196253,...,4.87,4.78,4.98,4.47,f,9,4,2,0,0.39
2,56074.0,https://www.airbnb.com/rooms/56074,2.025060e+13,01/07/2025,city scrape,Great space in historical San Rafael,This great apartment is located in one of the ...,Very traditional neighborhood with all service...,https://a0.muscache.com/pictures/3005118/60dac...,265650,...,4.98,4.94,4.76,4.79,f,1,1,0,0,0.48
3,67703.0,https://www.airbnb.com/rooms/67703,2.025060e+13,01/07/2025,city scrape,"2 bedroom apt. deco bldg, Condesa","Comfortably furnished, sunny, 2 bedroom apt., ...",No neighborhood overview provided,https://a0.muscache.com/pictures/3281720/6f078...,334451,...,4.94,4.92,4.98,4.92,f,2,2,0,0,0.30
4,70644.0,https://www.airbnb.com/rooms/70644,2.025060e+13,01/07/2025,city scrape,Beautiful light Studio Coyoacan- full equipped !,COYOACAN designer studio quiet & safe! well eq...,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,212109,...,4.96,4.98,4.96,4.92,f,3,2,1,0,0.81


In [122]:
# Mostrar columnas con solo 2 valores únicos
for col in df.columns:
    if df[col].nunique(dropna=False) == 2:
        print(f"{col}: {df[col].unique()}")

source: ['city scrape' 'previous scrape']
host_is_superhost: ['f' 't']
host_has_profile_pic: ['t' 'f']
host_identity_verified: ['t' 'f']
instant_bookable: ['f' 't']


In [123]:
# Convertir variables dicotómicas a 0 y 1
df['source_bin'] = df['source'].replace({'city scrape': 0, 'previous scrape': 1})
df['host_is_superhost_bin'] = df['host_is_superhost'].replace({'f': 0, 't': 1})
df['host_has_profile_pic_bin'] = df['host_has_profile_pic'].replace({'f': 0, 't': 1})
df['host_identity_verified_bin'] = df['host_identity_verified'].replace({'f': 0, 't': 1})
df['instant_bookable_bin'] = df['instant_bookable'].replace({'f': 0, 't': 1})


/tmp/ipython-input-2915928073.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['source_bin'] = df['source'].replace({'city scrape': 0, 'previous scrape': 1})
/tmp/ipython-input-2915928073.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['host_is_superhost_bin'] = df['host_is_superhost'].replace({'f': 0, 't': 1})
/tmp/ipython-input-2915928073.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy

**10 casos de correlación**

In [124]:
# Lista de modelos
modelos = [
    # Modelo 1: source_bin
    {'dependiente': 'source_bin', 'independientes': ['host_has_profile_pic_bin', 'longitude', 'minimum_nights_avg_ntm']},

    # Modelos 2–4: host_is_superhost_bin
    {'dependiente': 'host_is_superhost_bin', 'independientes': ['source_bin', 'review_scores_location', 'bedrooms']},
    {'dependiente': 'host_is_superhost_bin', 'independientes': ['instant_bookable_bin', 'review_scores_value', 'maximum_nights']},
    {'dependiente': 'host_is_superhost_bin', 'independientes': ['host_identity_verified_bin', 'longitude', 'review_scores_checkin']},

    # Modelos 5–7: host_identity_verified_bin
    {'dependiente': 'host_identity_verified_bin', 'independientes': ['source_bin', 'bedrooms', 'review_scores_location']},
    {'dependiente': 'host_identity_verified_bin', 'independientes': ['host_is_superhost_bin', 'minimum_nights_avg_ntm', 'maximum_nights']},
    {'dependiente': 'host_identity_verified_bin', 'independientes': ['instant_bookable_bin', 'review_scores_value', 'longitude']},

    # Modelos 8–10: instant_bookable_bin
    {'dependiente': 'instant_bookable_bin', 'independientes': ['source_bin', 'review_scores_value', 'maximum_nights']},
    {'dependiente': 'instant_bookable_bin', 'independientes': ['host_identity_verified_bin', 'bedrooms', 'review_scores_checkin']},
    {'dependiente': 'instant_bookable_bin', 'independientes': ['host_is_superhost_bin', 'longitude', 'minimum_nights_avg_ntm']},
]

# Ejecutar cada modelo
for i, modelo in enumerate(modelos, start=1):
    print(f"\n{'='*60}")
    print(f"Modelo {i}: Variable dependiente = {modelo['dependiente']}")
    print(f"Variables independientes: {modelo['independientes']}")

    # Definir X e y
    X = df[modelo['independientes']]
    y = df[modelo['dependiente']]

    # División en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Escalamiento
    escalar = StandardScaler()
    X_train_scaled = escalar.fit_transform(X_train)
    X_test_scaled = escalar.transform(X_test)

    # Entrenamiento del modelo
    modelo_log = LogisticRegression()
    modelo_log.fit(X_train_scaled, y_train)

    # Predicción
    y_pred = modelo_log.predict(X_test_scaled)

    # Coeficientes
    coeficientes = pd.DataFrame({
        'Variable': modelo['independientes'],
        'Coeficiente': modelo_log.coef_[0]
    })
    print("\nCoeficientes:")
    print(coeficientes)

    # Precisión por clase
    precision_clase = precision_score(y_test, y_pred, average=None, zero_division=0)
    print("\nPrecisión por clase:")
    print(precision_clase)

    # Precisión ponderada
    precision_ponderada = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    print("\nPrecisión ponderada:")
    print(precision_ponderada)

    # Matriz de confusión
    matriz = confusion_matrix(y_test, y_pred)
    print("\nMatriz de confusión:")
    print(matriz)



Modelo 1: Variable dependiente = source_bin
Variables independientes: ['host_has_profile_pic_bin', 'longitude', 'minimum_nights_avg_ntm']

Coeficientes:
                   Variable  Coeficiente
0  host_has_profile_pic_bin     0.160584
1                 longitude     0.077921
2    minimum_nights_avg_ntm     0.076811

Precisión por clase:
[0.87514203 0.        ]

Precisión ponderada:
0.7658735683349276

Matriz de confusión:
[[6932    0]
 [ 989    0]]

Modelo 2: Variable dependiente = host_is_superhost_bin
Variables independientes: ['source_bin', 'review_scores_location', 'bedrooms']

Coeficientes:
                 Variable  Coeficiente
0              source_bin    -0.427520
1  review_scores_location     0.094176
2                bedrooms     0.086626

Precisión por clase:
[0.63956571 0.        ]

Precisión ponderada:
0.409044299198685

Matriz de confusión:
[[5066    0]
 [2855    0]]

Modelo 3: Variable dependiente = host_is_superhost_bin
Variables independientes: ['instant_bookable_bin'

**3 casos de balanceo**

**source**

In [125]:
# Correlación con la variable dependiente
correlaciones = df.corr(numeric_only=True)['source_bin'].sort_values(ascending=False)
print(correlaciones.head(10))

source_bin                                      1.000000
host_has_profile_pic_bin                        0.040852
calculated_host_listings_count_private_rooms    0.034305
longitude                                       0.020303
minimum_nights_avg_ntm                          0.019972
review_scores_value                             0.013691
maximum_nights                                  0.011303
review_scores_location                          0.007389
bedrooms                                        0.006564
review_scores_checkin                           0.006479
Name: source_bin, dtype: float64


In [126]:
# 1. Selección de variables
Vars_Indep = df[['host_has_profile_pic_bin',
                 'calculated_host_listings_count_private_rooms', 'longitude' ]]
Var_Dep = df['source_bin']

# 2. Redefinir variables
X = Vars_Indep
y = Var_Dep

# 3. División en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# 4. Escalamiento de variables
from sklearn.preprocessing import StandardScaler
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# 5. Definición y entrenamiento del modelo
from sklearn.linear_model import LogisticRegression
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# 6. Predicción
y_pred = modelo.predict(X_test)

# 7. Coeficientes por variable
import pandas as pd
coeficientes = pd.DataFrame({
    'Variable': Vars_Indep.columns,
    'Coeficiente': modelo.coef_[0]
})
print("Coeficientes por variable:")
print(coeficientes)

# 8. Precisión por clase
from sklearn.metrics import precision_score, confusion_matrix
precision_por_clase = precision_score(y_test, y_pred, average=None, zero_division=0)
print("\nPrecisión por clase (0 = previous scrape, 1 = city scrape):")
print(precision_por_clase)

# 9. Precisión ponderada
precision_ponderada = precision_score(y_test, y_pred, average='weighted', zero_division=0)
print("\nPrecisión promedio ponderada:")
print(precision_ponderada)

# 10. Matriz de confusión
matriz = confusion_matrix(y_test, y_pred)
print("\nMatriz de confusión:")
print(matriz)


Coeficientes por variable:
                                       Variable  Coeficiente
0                      host_has_profile_pic_bin     0.146878
1  calculated_host_listings_count_private_rooms     0.092176
2                                     longitude     0.053211

Precisión por clase (0 = previous scrape, 1 = city scrape):
[0.8799394 0.       ]

Precisión promedio ponderada:
0.7742933504718138

Matriz de confusión:
[[6970    0]
 [ 951    0]]


In [127]:
# Aplicar SMOTE sobre el conjunto de entrenamiento ya escalado
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Entrenar el modelo con datos balanceados (sin class_weight)
modelo_smote = LogisticRegression()
modelo_smote.fit(X_train_resampled, y_train_resampled)

# Predicción sobre el conjunto de prueba original
y_pred_smote = modelo_smote.predict(X_test)

# Evaluación del modelo balanceado con SMOTE
coeficientes_smote = pd.DataFrame({
    'Variable': Vars_Indep.columns,
    'Coeficiente': modelo_smote.coef_[0]
})
print("\nCoeficientes del modelo con SMOTE:")
print(coeficientes_smote)

precision_por_clase_smote = precision_score(y_test, y_pred_smote, average=None, zero_division=0)
print("\nPrecisión por clase con SMOTE (0 = previous scrape, 1 = city scrape):")
print(precision_por_clase_smote)

precision_ponderada_smote = precision_score(y_test, y_pred_smote, average='weighted', zero_division=0)
print("\nPrecisión promedio ponderada con SMOTE:")
print(precision_ponderada_smote)

matriz_smote = confusion_matrix(y_test, y_pred_smote)
print("\nMatriz de confusión con SMOTE:")
print(matriz_smote)



Coeficientes del modelo con SMOTE:
                                       Variable  Coeficiente
0                      host_has_profile_pic_bin     0.136741
1  calculated_host_listings_count_private_rooms     0.100419
2                                     longitude     0.054845

Precisión por clase con SMOTE (0 = previous scrape, 1 = city scrape):
[0.898445   0.15560472]

Precisión promedio ponderada con SMOTE:
0.8092591505877075

Matriz de confusión con SMOTE:
[[4680 2290]
 [ 529  422]]


**Host is superhost**

In [128]:
# Correlación con la variable dependiente
correlaciones = df.corr(numeric_only=True)['host_is_superhost_bin'].sort_values(ascending=False)
print(correlaciones.head(10))

host_is_superhost_bin          1.000000
estimated_occupancy_l365d      0.351063
number_of_reviews_ltm          0.335287
number_of_reviews_ly           0.301968
number_of_reviews              0.263501
estimated_revenue_l365d        0.262718
reviews_per_month              0.217193
host_response_rate             0.216573
review_scores_rating           0.182102
review_scores_communication    0.170942
Name: host_is_superhost_bin, dtype: float64


In [129]:
# 1. Selección de variables
Vars_Indep = df[['estimated_occupancy_l365d', 'number_of_reviews_ltm', 'number_of_reviews_ly']]
Var_Dep = df['host_is_superhost_bin']

# 2. Redefinir variables
X = Vars_Indep
y = Var_Dep

# 3. División en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# 4. Escalamiento de variables
from sklearn.preprocessing import StandardScaler
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# 5. Definición y entrenamiento del modelo
from sklearn.linear_model import LogisticRegression
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# 6. Predicción
y_pred = modelo.predict(X_test)

# 7. Coeficientes por variable
import pandas as pd
coeficientes = pd.DataFrame({
    'Variable': Vars_Indep.columns,
    'Coeficiente': modelo.coef_[0]
})
print("Coeficientes por variable:")
print(coeficientes)

# 8. Precisión por clase
from sklearn.metrics import precision_score, confusion_matrix
precision_por_clase = precision_score(y_test, y_pred, average=None, zero_division=0)
print("\nPrecisión por clase (0 = no Superhost, 1 = sí Superhost):")
print(precision_por_clase)

# 9. Precisión ponderada
precision_ponderada = precision_score(y_test, y_pred, average='weighted', zero_division=0)
print("\nPrecisión promedio ponderada:")
print(precision_ponderada)

# 10. Matriz de confusión
matriz = confusion_matrix(y_test, y_pred)
print("\nMatriz de confusión:")
print(matriz)


Coeficientes por variable:
                    Variable  Coeficiente
0  estimated_occupancy_l365d     0.466547
1      number_of_reviews_ltm     0.146020
2       number_of_reviews_ly     0.219130

Precisión por clase (0 = no Superhost, 1 = sí Superhost):
[0.71200543 0.59005906]

Precisión promedio ponderada:
0.66743596454311

Matriz de confusión:
[[4193  833]
 [1696 1199]]


In [130]:
# Entrenar el modelo con ajuste de pesos para clases desbalanceadas
modelo_balanceado = LogisticRegression(class_weight='balanced')
modelo_balanceado.fit(X_train, y_train)

# Predicción sobre el conjunto de prueba original
y_pred_balanceado = modelo_balanceado.predict(X_test)

# Evaluación del modelo balanceado
coeficientes_balanceado = pd.DataFrame({
    'Variable': Vars_Indep.columns,
    'Coeficiente': modelo_balanceado.coef_[0]
})
print("\nCoeficientes del modelo con class_weight='balanced':")
print(coeficientes_balanceado)

precision_por_clase_balanceado = precision_score(y_test, y_pred_balanceado, average=None, zero_division=0)
print("\nPrecisión por clase con class_weight='balanced' (0 = no Superhost, 1 = sí Superhost):")
print(precision_por_clase_balanceado)

precision_ponderada_balanceado = precision_score(y_test, y_pred_balanceado, average='weighted', zero_division=0)
print("\nPrecisión promedio ponderada con class_weight='balanced':")
print(precision_ponderada_balanceado)

matriz_balanceado = confusion_matrix(y_test, y_pred_balanceado)
print("\nMatriz de confusión con class_weight='balanced':")
print(matriz_balanceado)



Coeficientes del modelo con class_weight='balanced':
                    Variable  Coeficiente
0  estimated_occupancy_l365d     0.475342
1      number_of_reviews_ltm     0.158719
2       number_of_reviews_ly     0.232996

Precisión por clase con class_weight='balanced' (0 = no Superhost, 1 = sí Superhost):
[0.76152591 0.57171249]

Precisión promedio ponderada con class_weight='balanced':
0.6921521101005612

Matriz de confusión con class_weight='balanced':
[[3733 1293]
 [1169 1726]]


**Instant bookable**

In [131]:
# Correlación con la variable dependiente
correlaciones = df.corr(numeric_only=True)['instant_bookable_bin'].sort_values(ascending=False)
print(correlaciones.head(10))

instant_bookable_bin                           1.000000
host_acceptance_rate                           0.262980
host_listings_count                            0.247481
calculated_host_listings_count                 0.243617
host_total_listings_count                      0.232683
id                                             0.186808
availability_365                               0.181300
calculated_host_listings_count_entire_homes    0.177277
maximum_maximum_nights                         0.169274
host_id                                        0.166093
Name: instant_bookable_bin, dtype: float64


In [132]:
# 1. Selección de variables
Vars_Indep = df[['host_acceptance_rate', 'host_listings_count',
                 'calculated_host_listings_count', 'host_total_listings_count']]
Var_Dep = df['instant_bookable_bin']

# 2. Redefinir variables
X = Vars_Indep
y = Var_Dep

# 3. División en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# 4. Escalamiento de variables
from sklearn.preprocessing import StandardScaler
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# 5. Definición y entrenamiento del modelo
from sklearn.linear_model import LogisticRegression
modelo = LogisticRegression(class_weight='balanced')
modelo.fit(X_train, y_train)

# 6. Predicción
y_pred = modelo.predict(X_test)

# 7. Coeficientes por variable
import pandas as pd
coeficientes = pd.DataFrame({
    'Variable': Vars_Indep.columns,
    'Coeficiente': modelo.coef_[0]
})
print("Coeficientes por variable:")
print(coeficientes)

# 8. Precisión por clase
from sklearn.metrics import precision_score, confusion_matrix
precision_por_clase = precision_score(y_test, y_pred, average=None, zero_division=0)
print("\nPrecisión por clase (0 = no instant booking, 1 = sí):")
print(precision_por_clase)

# 9. Precisión ponderada
precision_ponderada = precision_score(y_test, y_pred, average='weighted', zero_division=0)
print("\nPrecisión promedio ponderada:")
print(precision_ponderada)

# 10. Matriz de confusión
matriz = confusion_matrix(y_test, y_pred)
print("\nMatriz de confusión:")
print(matriz)


Coeficientes por variable:
                         Variable  Coeficiente
0            host_acceptance_rate     0.626093
1             host_listings_count     0.176154
2  calculated_host_listings_count     0.276357
3       host_total_listings_count     0.129060

Precisión por clase (0 = no instant booking, 1 = sí):
[0.778566   0.59335682]

Precisión promedio ponderada:
0.7029952293503864

Matriz de confusión:
[[3073 1616]
 [ 874 2358]]


In [133]:
# Aplicar SMOTE sobre el conjunto de entrenamiento ya escalado
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Entrenar un nuevo modelo sin class_weight (ya está balanceado por SMOTE)
modelo_smote = LogisticRegression()
modelo_smote.fit(X_train_resampled, y_train_resampled)

# Predicción sobre el conjunto de prueba original
y_pred_smote = modelo_smote.predict(X_test)

# Coeficientes del modelo con SMOTE
coeficientes_smote = pd.DataFrame({
    'Variable': Vars_Indep.columns,
    'Coeficiente': modelo_smote.coef_[0]
})
print("\nCoeficientes del modelo con SMOTE:")
print(coeficientes_smote)

# Precisión por clase
precision_por_clase_smote = precision_score(y_test, y_pred_smote, average=None, zero_division=0)
print("\nPrecisión por clase con SMOTE (0 = no instant booking, 1 = sí):")
print(precision_por_clase_smote)

# Precisión ponderada
precision_ponderada_smote = precision_score(y_test, y_pred_smote, average='weighted', zero_division=0)
print("\nPrecisión promedio ponderada con SMOTE:")
print(precision_ponderada_smote)

# Matriz de confusión
matriz_smote = confusion_matrix(y_test, y_pred_smote)
print("\nMatriz de confusión con SMOTE:")
print(matriz_smote)



Coeficientes del modelo con SMOTE:
                         Variable  Coeficiente
0            host_acceptance_rate     0.626865
1             host_listings_count     0.175132
2  calculated_host_listings_count     0.278014
3       host_total_listings_count     0.129800

Precisión por clase con SMOTE (0 = no instant booking, 1 = sí):
[0.778566   0.59335682]

Precisión promedio ponderada con SMOTE:
0.7029952293503864

Matriz de confusión con SMOTE:
[[3073 1616]
 [ 874 2358]]


**Conversión a dicotómicas**

Baños (bathrooms)

In [134]:
#Obtenemos el limite superior y el límite inferior de la columna 'bathrooms'
Max=df['bathrooms'].max()
Min=df['bathrooms'].min()
Limites= [Min, Max]
Limites

[0.0, 3.5]

In [135]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos=np.linspace(0.0, 3.5, 3)
intervalos

#Creamos las categorías
categorias= ['Un baño', 'Varios baños']

# Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['bathrooms_cat']=pd.cut(x= df['bathrooms'], bins=intervalos, labels= categorias )

# Imputar los NAs con moda
moda = df['bathrooms_cat'].mode()[0]
df['bathrooms_cat'] = df['bathrooms_cat'].fillna(moda)

In [136]:
df['bathrooms_cat'].head(5)

,bathrooms_cat
0,Un baño
1,Un baño
2,Un baño
3,Un baño
4,Un baño


In [137]:
df['bathrooms_bin'] = (df['bathrooms_cat'] == 'Varios baños').astype(int)
df['bathrooms_bin'].head(5)

,bathrooms_bin
0,0
1,0
2,0
3,0
4,0


Habitaciones (bedrooms)

In [138]:
#Obtenemos el limite superior y el límite inferior de la columna 'bedrooms'
Max=df['bedrooms'].max()
Min=df['bedrooms'].min()
Limites= [Min, Max]
Limites

[0, 3]

In [139]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos=np.linspace(0.0, 3.5, 3)
intervalos

#Creamos las categorías
categorias= ['Una recámara', 'Varias recámaras']

# Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['bedrooms_cat']=pd.cut(x= df['bedrooms'], bins=intervalos, labels= categorias )

# Imputar los NAs con moda
moda = df['bedrooms_cat'].mode()[0]
df['bedrooms_cat'] = df['bedrooms_cat'].fillna(moda)

In [140]:
df['bedrooms_cat'].head(5)

,bedrooms_cat
0,Una recámara
1,Una recámara
2,Una recámara
3,Varias recámaras
4,Una recámara


In [141]:
df['bedrooms_bin'] = (df['bedrooms_cat'] == 'Varias recámaras').astype(int)
df['bedrooms_bin'].head(5)

,bedrooms_bin
0,0
1,0
2,0
3,1
4,0


Minimo de noches (minimum_nights)

In [142]:
#Obtenemos el limite superior y el límite inferior de la columna 'minimum_nights'
Max=df['minimum_nights'].max()
Min=df['minimum_nights'].min()
Limites= [Min, Max]
Limites

[1, 3]

In [143]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos = [0.5, 1.5, 3.1]
categorias = ['Una noche', 'Dos o más noches']
df['min_nights_cat'] = pd.cut(
    df['minimum_nights'],
    bins=intervalos,
    labels=categorias,
    include_lowest=True,
    right=False  # Cierra el intervalo por la izquierda: [1,2), [2,3.1)
)

# Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['min_nights_cat']=pd.cut(x= df['minimum_nights'], bins=intervalos, labels= categorias )

# Imputar los NAs con moda
moda = df['min_nights_cat'].mode()[0]
df['min_nights_cat'] = df['min_nights_cat'].fillna(moda)

In [144]:
df['min_nights_cat'].head(5)

,min_nights_cat
0,Una noche
1,Una noche
2,Una noche
3,Dos o más noches
4,Dos o más noches


In [145]:
df['min_nights_bin'] = (df['min_nights_cat'] == 'Dos o más noches').astype(int)
df['min_nights_bin'].head(5)

,min_nights_bin
0,0
1,0
2,0
3,1
4,1


Calificación del alojamiento (review_scores_value)

In [146]:
#Obtenemos el limite superior y el límite inferior de la columna 'review_scores_value'
Max=df['review_scores_value'].max()
Min=df['review_scores_value'].min()
Limites= [Min, Max]
Limites

[4.37, 5.0]

In [151]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos = [4.37, 4.8, 5.0]
categorias = ['Buena', 'Excelente']

df['review_scores_val_cat'] = pd.cut(
    x=df['review_scores_value'],
    bins=intervalos,
    labels=categorias,
    include_lowest=True
)
#Creamos las categorías
categorias= ['Buena', 'Excelente']

# Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['review_scores_val_cat']=pd.cut(x= df['review_scores_value'], bins=intervalos, labels= categorias )

# Imputar los NAs con moda
moda = df['review_scores_val_cat'].mode()[0]
df['review_scores_val_cat'] = df['review_scores_val_cat'].fillna(moda)

In [153]:
df['review_scores_val_cat'].head(5)

,review_scores_val_cat
0,Buena
1,Buena
2,Buena
3,Excelente
4,Excelente


In [154]:
df['review_scores_val_bin'] = (df['review_scores_val_cat'] == 'Excelente').astype(int)
df['review_scores_val_bin'].head(5)

,review_scores_val_bin
0,0
1,0
2,0
3,1
4,1


Capacidad de huespedes (accommodates)

In [155]:
#Obtenemos el limite superior y el límite inferior de la columna 'accommodates'
Max=df['accommodates'].max()
Min=df['accommodates'].min()
Limites= [Min, Max]
Limites

[1, 7]

In [183]:
df['accommodates_cat'] = df['accommodates'].apply(
    lambda x: 'Individual o 2 personas' if x <= 2 else '3 o más personas'
)

# Rellenar NAs con moda
moda = df['accommodates_cat'].mode()[0]
df['accommodates_cat'] = df['accommodates_cat'].fillna(moda)  # ← CORRECCIÓN CLAVE

print(df['accommodates_cat'].value_counts())

accommodates_cat
Individual o 2 personas    14874
3 o más personas           11527
Name: count, dtype: int64


In [189]:
df['accommodates_cat'].head(5)

,accommodates_cat
0,Individual o 2 personas
1,Individual o 2 personas
2,Individual o 2 personas
3,3 o más personas
4,Individual o 2 personas


In [188]:
df['accommodates_bin'] = (df['accommodates_cat'] == '3 o más personas').astype(int)
df['accommodates_bin'].head(5)

,accommodates_bin
0,0
1,0
2,0
3,1
4,0


In [191]:
df.isnull().sum()

,0
id,0
listing_url,0
scrape_id,0
last_scraped,0
source,0
name,0
description,0
neighborhood_overview,0
picture_url,0
host_id,0


**3 modelos con las variables convertidas**

bedrooms

In [192]:
correlaciones = df.corr(numeric_only=True)['bedrooms_bin'].sort_values(ascending=False)
print(correlaciones.head(10))

bedrooms_bin               1.000000
bedrooms                   0.907427
accommodates               0.680324
accommodates_bin           0.631859
bathrooms_bin              0.560393
beds                       0.516491
bathrooms                  0.479389
price                      0.374879
estimated_revenue_l365d    0.157177
maximum_minimum_nights     0.132787
Name: bedrooms_bin, dtype: float64


In [196]:
# 1. Selección de variables
Vars_Indep = df[['accommodates',
                 'beds','bathrooms', 'price' ]]
Var_Dep = df['bedrooms_bin']

# 2. Redefinir variables
X = Vars_Indep
y = Var_Dep

# 3. División en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# 4. Escalamiento de variables
from sklearn.preprocessing import StandardScaler
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# 5. Definición y entrenamiento del modelo
from sklearn.linear_model import LogisticRegression
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# 6. Predicción
y_pred = modelo.predict(X_test)

# 7. Coeficientes por variable
import pandas as pd
coeficientes = pd.DataFrame({
    'Variable': Vars_Indep.columns,
    'Coeficiente': modelo.coef_[0]
})
print("Coeficientes por variable:")
print(coeficientes)

# 8. Precisión por clase
from sklearn.metrics import precision_score, confusion_matrix
precision_por_clase = precision_score(y_test, y_pred, average=None, zero_division=0)
print("\nPrecisión por clase (0 = una habitación, 1 = varias habitaciones):")
print(precision_por_clase)

# 9. Precisión ponderada
precision_ponderada = precision_score(y_test, y_pred, average='weighted', zero_division=0)
print("\nPrecisión promedio ponderada:")
print(precision_ponderada)

# 10. Matriz de confusión
matriz = confusion_matrix(y_test, y_pred)
print("\nMatriz de confusión:")
print(matriz)


Coeficientes por variable:
       Variable  Coeficiente
0  accommodates     1.615645
1          beds     0.663847
2     bathrooms     0.934020
3         price     0.383531

Precisión por clase (0 = una habitación, 1 = varias habitaciones):
[0.89987303 0.82931894]

Precisión promedio ponderada:
0.8771685233225229

Matriz de confusión:
[[4961  411]
 [ 552 1997]]


accommodates

In [197]:
correlaciones = df.corr(numeric_only=True)['accommodates_bin'].sort_values(ascending=False)
print(correlaciones.head(10))

accommodates_bin             1.000000
accommodates                 0.854330
bedrooms_bin                 0.631859
beds                         0.591409
bedrooms                     0.540688
bathrooms_bin                0.355825
price                        0.348647
bathrooms                    0.289815
estimated_revenue_l365d      0.201557
estimated_occupancy_l365d    0.128275
Name: accommodates_bin, dtype: float64


In [200]:
# 1. Selección de variables
Vars_Indep = df[['bedrooms_bin',
                 'beds', 'price' ]]
Var_Dep = df['accommodates_bin']

# 2. Redefinir variables
X = Vars_Indep
y = Var_Dep

# 3. División en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# 4. Escalamiento de variables
from sklearn.preprocessing import StandardScaler
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# 5. Definición y entrenamiento del modelo
from sklearn.linear_model import LogisticRegression
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# 6. Predicción
y_pred = modelo.predict(X_test)

# 7. Coeficientes por variable
import pandas as pd
coeficientes = pd.DataFrame({
    'Variable': Vars_Indep.columns,
    'Coeficiente': modelo.coef_[0]
})
print("Coeficientes por variable:")
print(coeficientes)

# 8. Precisión por clase
from sklearn.metrics import precision_score, confusion_matrix
precision_por_clase = precision_score(y_test, y_pred, average=None, zero_division=0)
print("\nPrecisión por clase (0 = Individual o 2 personas, 1 = 3 o más personas):")
print(precision_por_clase)

# 9. Precisión ponderada
precision_ponderada = precision_score(y_test, y_pred, average='weighted', zero_division=0)
print("\nPrecisión promedio ponderada:")
print(precision_ponderada)

# 10. Matriz de confusión
matriz = confusion_matrix(y_test, y_pred)
print("\nMatriz de confusión:")
print(matriz)


Coeficientes por variable:
       Variable  Coeficiente
0  bedrooms_bin     1.202212
1          beds     1.496076
2         price     0.415744

Precisión por clase (0 = Individual o 2 personas, 1 = 3 o más personas):
[0.8732982  0.86486486]

Precisión promedio ponderada:
0.8695835289974151

Matriz de confusión:
[[3977  455]
 [ 577 2912]]


review_scores_value

In [201]:
correlaciones = df.corr(numeric_only=True)['review_scores_val_bin'].sort_values(ascending=False)
print(correlaciones.head(10))

review_scores_val_bin          1.000000
review_scores_value            0.831395
review_scores_rating           0.617557
review_scores_accuracy         0.564941
review_scores_cleanliness      0.487598
review_scores_communication    0.483622
review_scores_checkin          0.452466
review_scores_location         0.334747
host_is_superhost_bin          0.197325
min_nights_bin                 0.111340
Name: review_scores_val_bin, dtype: float64


In [204]:
# 1. Selección de variables
Vars_Indep = df[['review_scores_rating',
                 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_communication' ]]
Var_Dep = df['review_scores_val_bin']

# 2. Redefinir variables
X = Vars_Indep
y = Var_Dep

# 3. División en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# 4. Escalamiento de variables
from sklearn.preprocessing import StandardScaler
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# 5. Definición y entrenamiento del modelo
from sklearn.linear_model import LogisticRegression
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# 6. Predicción
y_pred = modelo.predict(X_test)

# 7. Coeficientes por variable
import pandas as pd
coeficientes = pd.DataFrame({
    'Variable': Vars_Indep.columns,
    'Coeficiente': modelo.coef_[0]
})
print("Coeficientes por variable:")
print(coeficientes)

# 8. Precisión por clase
from sklearn.metrics import precision_score, confusion_matrix
precision_por_clase = precision_score(y_test, y_pred, average=None, zero_division=0)
print("\nPrecisión por clase (0 = Buena, 1 = Excelente):")
print(precision_por_clase)

# 9. Precisión ponderada
precision_ponderada = precision_score(y_test, y_pred, average='weighted', zero_division=0)
print("\nPrecisión promedio ponderada:")
print(precision_ponderada)

# 10. Matriz de confusión
matriz = confusion_matrix(y_test, y_pred)
print("\nMatriz de confusión:")
print(matriz)


Coeficientes por variable:
                      Variable  Coeficiente
0         review_scores_rating     1.086758
1       review_scores_accuracy     0.719416
2    review_scores_cleanliness     0.409810
3  review_scores_communication     0.484891

Precisión por clase (0 = Buena, 1 = Excelente):
[0.83664516 0.84008898]

Precisión promedio ponderada:
0.8383981551045732

Matriz de confusión:
[[3242  647]
 [ 633 3399]]


In [ ]:
# Para este tarbajo se utilizarón mis ideas, los códigos vistos en clase y consultas de IA